In [276]:
#Fundamental matrix estimation. 

In [277]:

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


In [278]:
def normalize(img):
    ''' Function to normalize an input array to 0-1 '''
    img_min = img.min()
    img_max = img.max()
    return (img - img_min) / (img_max - img_min)

In [279]:
def coor_to_homo(coord):
    height = len(coord)
    new_col = np.ones((height,1))
    homo = np.hstack((coord,new_col))
    return homo


In [280]:
def my_normalize(matches):
    mean_num = np.mean(matches,axis=0)
    m_off = np.eye(3,dtype=float)
    m_off[0][2],m_off[1][2] = -mean_num[0],-mean_num[1]
    m_scale= np.eye(3,dtype=float)
    m_scale[0, 0] = 1.0 / max(abs(matches[:, 0]))
    m_scale[1, 1] = 1.0 / max(abs(matches[:, 1]))
    coor_trans = np.matmul(m_scale,m_off)
    noralized_match = np.transpose(np.matmul(coor_trans,
                                             np.transpose(matches)))
    
    return coor_trans,noralized_match

def fit_fundamental(matches,method):
    match1 = coor_to_homo(matches[:, 0:2])
    match2 = coor_to_homo(matches[:, 2:4])
    print(match1.shape)
    # print(match1)
    if method == "normalized":
        coor_trans1,match1 = my_normalize(match1)
        coor_trans2,match2 = my_normalize(match2)
        
    else:
        pass
    num_of_matches  = len(matches)
    u = match1[:, 0]
    v = match1[:, 1]
    u_p = match2[:, 0]
    v_p = match2[:, 1]
    
    A = np.multiply(u_p,u)
    line_list = [
                np.multiply(u_p,v),
                u_p,
                np.multiply(v_p,u),
                np.multiply(v_p,v),
                v_p,
                u,
                v,
                np.ones(num_of_matches)]
    
    for i in range(len(line_list)):
        A = np.vstack((A,line_list[i]))
        
    A = np.reshape(A,(-1,9))
    print("A shape:",A.shape)
    _,_,V = np.linalg.svd(A)
    print("V shape:",V.shape)
    F = V[len(V)-1]
    
    F = np.reshape(F,(3,3))
    U,S,V = np.linalg.svd(F)
    print(U.shape,S.shape,V.shape)
    S[-1] = 0
    F = np.matmul(U * S[..., None, :],np.transpose(V) )
    print("F shape",F.shape)
    if method == "normalized":
        F = np.matmul(np.transpose(coor_trans2),F)
        F = np.matmul(F,coor_trans1)
        
    print("F:",F)
    residul = []
    for i in range(num_of_matches):
        tmp_m = np.array([matches[i][0],matches[i][1],1])
        tmp_m = np.transpose(tmp_m)
        X = np.matmul(F, tmp_m)
        # print("X shape:",X.shape)
        residul += [np.abs(np.matmul(
            np.array([matches[i][2],matches[i][3],1]),X
                                     ))]
    print("residul sum:",np.sum(residul))
    return F

In [281]:
lb1path = "data\\part2\\library1.jpg"
lb2path = "data\\part2\\library2.jpg"
lb_matches = "data\\part2\\library_matches.txt"


In [284]:

##
## load images and match files for the first example
##

I1 = Image.open(lb1path)
I2 = Image.open(lb2path)
matches = np.loadtxt(lb_matches)

# this is a N x 4 file where the first two numbers of each row
# are coordinates of corners in the first image and the last two
# are coordinates of corresponding corners in the second image: 
# matches(i,1:2) is a point in the first image
# matches(i,3:4) is a corresponding point in the second image

N = len(matches)

##
## display two images side-by-side with matches
## this code is to help you visualize the matches, you don't need
## to use it to produce the results for the assignment
##

I3 = np.zeros((I1.size[1],I1.size[0]*2,3) )
I3[:,:I1.size[0],:] = I1
I3[:,I1.size[0]:,:] = I2
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.imshow(np.array(normalize(I3)).astype(float))
ax.plot(matches[:,0],matches[:,1],  '+r')
ax.plot( matches[:,2]+I1.size[0],matches[:,3], '+r')
ax.plot([matches[:,0], matches[:,2]+I1.size[0]],[matches[:,1], matches[:,3]], 'r')
plt.show()


In [286]:

##
## display second image with epipolar lines reprojected 
## from the first image
##

# first, fit fundamental matrix to the matches
F = fit_fundamental(matches,"unnormalized") # this is a function that you should write
M = np.c_[matches[:,0:2], np.ones((N,1))].transpose()
print("M shape:",M.shape)
L1 = np.matmul(F, M).transpose() # transform points from 
# the first image to get epipolar lines in the second image

# find points on epipolar lines L closest to matches(:,3:4)
l = np.sqrt(L1[:,0]**2 + L1[:,1]**2)
L = np.divide(L1,np.kron(np.ones((3,1)),l).transpose())# rescale the line
pt_line_dist = np.multiply(L, np.c_[matches[:,2:4], np.ones((N,1))]).sum(axis = 1)
closest_pt = matches[:,2:4] - np.multiply(L[:,0:2],np.kron(np.ones((2,1)), pt_line_dist).transpose())

# find endpoints of segment on epipolar line (for display purposes)
pt1 = closest_pt - np.c_[L[:,1], -L[:,0]]*10# offset from the closest point is 10 pixels
pt2 = closest_pt + np.c_[L[:,1], -L[:,0]]*10

# display points and segments of corresponding epipolar lines
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.imshow(normalize(np.array(I2).astype(float)))
ax.plot(matches[:,2],matches[:,3],  '+r')
ax.plot([matches[:,2], closest_pt[:,0]],[matches[:,3], closest_pt[:,1]], 'r')
ax.plot([pt1[:,0], pt2[:,0]],[pt1[:,1], pt2[:,1]], 'g')
plt.show()



(309, 3)
A shape: (309, 9)
V shape: (9, 9)
(3, 3) (3,) (3, 3)
F shape (3, 3)
F: [[ 0.29993401 -0.09611818 -0.06599279]
 [ 0.16460074 -0.05893679  0.52712223]
 [-0.72054415  0.23373247 -0.09852012]]
residul sum: 9700490.226475574
M shape: (3, 309)


In [287]:
#Camera calibration

In [288]:
def evaluate_points(M, points_2d, points_3d):
    """
    Visualize the actual 2D points and the projected 2D points calculated from
    the projection matrix
    You do not need to modify anything in this function, although you can if you
    want to
    :param M: projection matrix 3 x 4
    :param points_2d: 2D points N x 2
    :param points_3d: 3D points N x 3
    :return:
    """
    N = len(points_3d)
    points_3d = np.hstack((points_3d, np.ones((N, 1))))
    points_3d_proj = np.dot(M, points_3d.T).T
    u = points_3d_proj[:, 0] / points_3d_proj[:, 2]
    v = points_3d_proj[:, 1] / points_3d_proj[:, 2]
    residual = np.sum(np.hypot(u-points_2d[:, 0], v-points_2d[:, 1]))
    points_3d_proj = np.hstack((u[:, np.newaxis], v[:, np.newaxis]))
    return points_3d_proj, residual

In [289]:
def calculate_projection_matrix( Point_2D, Points_3D ):
    num_of_points = Points_3D.shape[0]
    print(num_of_points)
    A = np.array([])
    B = np.array([])
    M = np.zeros((11,1))
    for i in range(num_of_points):
        x = Points_3D[i][0]
        y = Points_3D[i][1]
        z = Points_3D[i][2]
        u = Point_2D[i][0]
        v = Point_2D[i][1]
        line_a = np.array([x,y,z,1,0,0,0,0,-u*x,-u*y,-u*z])
        A = np.vstack((A,line_a)) if len(A)>0 else line_a
        line_b = np.array([0, 0, 0, 0, x, y, z, 1 ,-v*x, -v*y, -v*z])
        A = np.vstack((A,line_b))
        B = np.append(B,u)
        B = np.append(B,v)
    M = np.array(np.linalg.lstsq(A,B)[0])
    M = np.append(M,1)
    M = np.reshape(M,(3,-1))
    
    return M
    
def compute_camera_center(M):
    left = -(((M[:, 0:3])))
    Center = np.linalg.lstsq(left, M[:, 3])[0]
    return Center


In [290]:
lab_matchtxt = "data\\part2\\lab_matches.txt"
lab_3d_path = "data\\part2\\lab_3d.txt"

point2D = np.loadtxt(lab_matchtxt)
point3D = np.loadtxt(lab_3d_path)
point2D_1 = np.array([[match[0],match[1]] for match in point2D])
point2D_2 = np.array([[match[2],match[3]] for match in point2D])
# print(point2D_1)
# print(point3D)
projection_M = calculate_projection_matrix(point2D_2, point3D)
print("M:",projection_M)
Center = compute_camera_center(projection_M)
print("Center:",Center)
 # 记得都要测
  
points3d_eval, residul = evaluate_points(projection_M, point2D_2, point3D)
print("residual：",residul)

20
M: [[-2.04662532e+00  1.18743052e+00  3.88938200e-01  2.43732985e+02]
 [-4.56886722e-01 -3.02017128e-01  2.14721848e+00  1.65932475e+02]
 [-2.24678720e-03 -1.09380146e-03  5.58547111e-04  1.00000000e+00]]
Center: [303.09666406 307.18423708  30.4222733 ]
residual： 15.621732317328656


C:\Users\bluerain\PycharmProjects\cvmp2\new_venv\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
C:\Users\bluerain\PycharmProjects\cvmp2\new_venv\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [ ]:
#Calculate the camera centers  and triangulate

In [333]:

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
C1_path = "data\\part2\\c1.txt"
C2_path = "data\\part2\\c2.txt"

def triangulate(P1_path, P2_path, matches_path):
    P1 = np.loadtxt(P1_path)
    P2 = np.loadtxt(P2_path)
    matches = np.loadtxt(matches_path)
    num_of_matches = matches.shape[0]
    # camera center, null space of projection matrix
    _,_,V1 = np.linalg.svd(P1)
    C1 = V1[len(V1)-2]

    _,_,V2 = np.linalg.svd(P2)
    C2 = V2[len(V2)-2]
 
    coor_3d = np.zeros((num_of_matches, 3))
    for i in range(num_of_matches):
        x1= np.array([
            [0, -1, matches[i][1]],
            [1, 0 , -matches[i][0]],
            [-matches[i][1],matches[i][0],0]
        ])
        x2 = np.array([
            [0,-1,matches[i][3]],
            [1, 0, -matches[i][2]],
            [-matches[i][3], matches[i][2], 0]
        ])
        D = np.vstack((np.matmul(x1,P1),np.matmul(x2,P2)))
        _,_,V = np.linalg.svd(D)
        X = V[len(V)-1]
        
        coor_3d[i,0:3] = [X[0]/X[3], X[1]/X[3], X[2]/X[3]]
  
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(coor_3d[:,0],coor_3d[:,1],coor_3d[:,2],marker='o',c='b')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
   
    ax.scatter(C1[0],C1[1],C1[2],marker='o',c='r')
    
    ax.text(C1[0],C1[1],C1[2], "C1", color='red')
    ax.scatter(C2[0],C2[1],C2[2],marker='o',c='r')
    
    ax.text(C2[0],C2[1]-0.6,C2[2], "C2", color='red')
    plt.show()
    # print(coor_3d)
    
    residual_1 = 0
    residual_2 = 0
    
    for i in range(num_of_matches):
        coor_tmp = np.append(coor_3d[i,:],1)
        X1 = np.matmul(P1,np.transpose(coor_tmp))
        residual_1 += residual_1 + np.sqrt(np.linalg.norm((np.array([X1[0]/X1[2],X1[1]/X1[2]]) -
                                                              np.array([matches[i, 0],matches[i, 1]]))))
        X2 = np.matmul(P2,np.transpose(coor_tmp))
        residual_2 += residual_2 + np.sqrt(np.linalg.norm((np.array([X1[0]/X1[2],X1[1]/X1[2]]) -
                                                              np.array([matches[i, 2],matches[i, 3]]))))        
    print("resiual1:",residual_1)
    print("residual2:",residual_2)

In [334]:

cam_data1 = "data\\part2\\library1_camera.txt"
cam_data2 = "data\\part2\\library2_camera.txt"
cam_match = "data\\part2\\library_matches.txt"
triangulate(cam_data1, cam_data2, cam_match)


V1: [[ 0.06228013  0.08470422  0.02113133  0.9942333 ]
 [-0.56147304  0.40411432  0.72196013 -0.01460176]
 [-0.78549604 -0.5222165  -0.31654564  0.10042287]
 [-0.25271561  0.74619484 -0.61491942 -0.03467258]]
V2: [[ 0.05582312  0.08427984 -0.00352149  0.99487099]
 [-0.59468418  0.57976812  0.55680328 -0.01377551]
 [-0.76565456 -0.610399   -0.17984341  0.09403455]
 [ 0.23876263 -0.53308432  0.81093406  0.03463312]]
C1: [-0.78549604 -0.5222165  -0.31654564  0.10042287]
c2: [-0.76565456 -0.610399   -0.17984341  0.09403455]
resiual1: 8.911574381637649e+92
residual2: 7.007290267804588e+93
